In [21]:
import numpy as np
import numpy.lib.stride_tricks
import math

In [14]:
u = np.zeros((2,2,10))
w = np.zeros((1,2,10))

In [25]:
#Custom Functions

#Trapezoid Integration Function
def integrate(u,dx,x_vals):
    area_under_curve = 0
    for i in range(0,len(x_vals)-1):
        area = dx * (u[i+1]+u[i]) / 2
        area_under_curve = area_under_curve + area
    return area_under_curve

#Gaussian Function
def gaussian(x, mu, sigma):
    denom = sigma * ((2 * math.pi)**0.5)
    numerator = math.exp(((x - mu)**2) / ((sigma) **2) / -2)
    res = numerator / denom
    return res

# Habitat Preference
def preference(x):
    pi = math.pi
    res = -math.cos(pi*x/5) + 1.1 
    res = np.float64(res)
    return res

# 1st Derivative of Habitat Preference Function
def preference_slope(x):
    pi = math.pi
    res = pi/5*math.sin(pi*x/5)
    res = np.float64(res)
    return res

# 2nd Derivative of Habitat Preference Function
def preference_slope_slope(x):
    pi = math.pi
    res = (2*pi*pi/625) * math.cos(2*pi*x/25)# Example 1
    #res = -0.02 * ((-0.02 * math.exp(-0.01*(x-50)**2)*(x-50))*(x-50)+1*math.exp(-0.01*(x-50)**2)) # Example 2
    res = np.float64(res)
    return res

#Normalized RMSE

def rmse(model, true, nx, t):
    summation = 0
    for i in range(0,nx):
        dif = (model[t][1][i] - true[0][1][i])**2
        summation = summation + dif
    rmse = (summation/nx)**0.5
    return abs(rmse)

#Convergence Flag

def converge(current_t, t_minus1000, t_minus2000):
    #function to determine whether finite difference scheme has convered
    #inputs are rows of an array
    current_t = np.float128(current_t ** 0.01)
    t_minus50 = np.float128(t_minus1000 ** 0.01)
    t_minus100 = np.float128(t_minus2000 ** 0.01)
    h1 = abs(t_minus1000 - t_minus2000)
    print(h1)
    h2 = abs(current_t - t_minus1000)
    print(h2)
    dif = abs(h1-h2)
    print(dif)
    cuttoff = np.format_float_scientific(np.float128(1e-128), unique=False, precision=128)
    mask = dif>np.float128(cuttoff)
    print(mask)
    if True not in mask:
        print("yes")
        return True
    
#Sliding window
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
        
    

In [45]:
# Initializing x values where u(x,t) will be calculated
dx = 1
Xs = np.arange(0, 11, dx)

# Initializing array with Nt row, Nx columns, and [x,u(x,t)] per cell
u = np.zeros((2, 2, len(Xs)))

#Habitat Preference Function
w = np.zeros((1,2,len(Xs)))
w[0][0] = Xs
for i in range(0,len(Xs)):
    w[0][1][i] = preference(w[0][0][i])

#First Spatial Derivative of Habitat Preference Function
wx = np.zeros((1,2,len(Xs)))
wx[0][0] = Xs
for i in range(0,len(Xs)):
    wx[0][1][i] = preference_slope(w[0][0][i])

#Second Spatial Derivative of Habitat Preference Function
wxx = np.zeros((1,2,len(Xs)))
wxx[0][0] = Xs
for i in range(0,len(Xs)):
    wxx[0][1][i] = preference_slope_slope(w[0][0][i])
    
# Populating x values in array u
for j in range(0, 2):
    u[j][0]=Xs
    
# Setting Initial Condition
IC=[1,2,3,4,5,6,5,4,3,2,1]
u[0][1] = IC # Populating initial condition in array u

In [41]:
print(u)
print(w)

[[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
  [ 1.  2.  3.  4.  5.  6.  5.  4.  3.  2.  1.]]

 [[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]]
[[[ 0.          1.          2.          3.          4.
    5.          6.          7.          8.          9.
   10.        ]
  [ 0.1         0.29098301  0.79098301  1.40901699  1.90901699
    2.1         1.90901699  1.40901699  0.79098301  0.29098301
    0.1       ]]]


In [76]:

advection_multipliers = np.array([-1,0,1])
diffusion_multipliers = np.array([1,-2,1])
u_win = rolling_window(u[0][1],3)
w_win = rolling_window(w[0][1],3)

advec_broad = np.broadcast(u_win,w_win,advection_multipliers)
advec = np.empty(advec_broad.shape)
advec.flat = [-t*u*v/(2*dx) for (t,u,v) in advec_broad]

dif_broad = np.broadcast(u_win,diffusion_multipliers)
dif = np.empty(dif_broad.shape)
dif.flat = [u*v/(2*dx) for (u,v) in dif_broad]


In [77]:
advec

array([[ 0.05      , -0.        , -1.18647451],
       [ 0.29098301, -0.        , -2.81803399],
       [ 1.18647451, -0.        , -4.77254249],
       [ 2.81803399, -0.        , -6.3       ],
       [ 4.77254249, -0.        , -4.77254249],
       [ 6.3       , -0.        , -2.81803399],
       [ 4.77254249, -0.        , -1.18647451],
       [ 2.81803399, -0.        , -0.29098301],
       [ 1.18647451, -0.        , -0.05      ]])

In [66]:
u_win

array([[1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.],
       [4., 5., 6.],
       [5., 6., 5.],
       [6., 5., 4.],
       [5., 4., 3.],
       [4., 3., 2.],
       [3., 2., 1.]])

In [67]:
w_win

array([[0.1       , 0.29098301, 0.79098301],
       [0.29098301, 0.79098301, 1.40901699],
       [0.79098301, 1.40901699, 1.90901699],
       [1.40901699, 1.90901699, 2.1       ],
       [1.90901699, 2.1       , 1.90901699],
       [2.1       , 1.90901699, 1.40901699],
       [1.90901699, 1.40901699, 0.79098301],
       [1.40901699, 0.79098301, 0.29098301],
       [0.79098301, 0.29098301, 0.1       ]])